# Setting Workspace

In [ ]:
!pip install ipython-autotime
!pip install transformers

In [ ]:
%load_ext autotime

time: 118 µs (started: 2021-08-09 08:50:59 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 3.06 ms (started: 2021-08-09 08:50:59 +00:00)


In [ ]:
!pwd
import os
os.chdir('/content/drive/My Drive/HLT/')
!pwd

/content
/content/drive/My Drive/HLT
time: 241 ms (started: 2021-08-09 08:50:59 +00:00)


# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, pipeline, TFBertModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from tensorflow.keras.layers import InputLayer, Embedding, Concatenate, TimeDistributed, Bidirectional, GRU
from tensorflow.keras.layers import LSTM, Dense,Flatten,Conv2D,Conv1D,GlobalMaxPooling1D,GlobalMaxPool1D
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

time: 12.7 s (started: 2021-08-09 08:50:59 +00:00)


In [ ]:
pd.set_option('display.max_colwidth', -1) #show all text in col
pd.set_option('display.max_rows', None) #show all rows

time: 3.47 ms (started: 2021-08-09 08:51:12 +00:00)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


# Create Bert Embeddings

In [ ]:
train_df = pd.read_csv("datasets/train.csv") 
train_df['headline']=train_df['headline'].apply(str)
train_df.head()

,headline,is_sarcastic
0,bill de blasio thinks proved haters wrong comes pre k,0
1,baby realize white supremacist yet,1
2,spring sprung arctic way early,0
3,mayor daley son appointed head illinois nepotist party,1
4,poll grandsons talented,1


time: 994 ms (started: 2021-08-09 07:47:43 +00:00)


In [ ]:
test_df = pd.read_csv("datasets/test.csv") 
test_df['headline']=test_df['headline'].apply(str)
test_df.head()

,headline,is_sarcastic
0,jessie james decker shares inspiring message post baby bodies,0
1,frustrated russian officials struggling get policies dysfunctional trump administration,1
2,second saturday staten island art walk,0
3,chris christie emits loud sob paul ryan asks crowd whether worse years ago,1
4,chicago police department monitor interactions public using new bullet cams,1


time: 412 ms (started: 2021-08-09 07:47:44 +00:00)


In [ ]:
print("Train size:{}".format(train_df.shape))
print("Test size:{}".format(test_df.shape))

Train size:(22702, 2)
Test size:(4007, 2)
time: 3.18 ms (started: 2021-08-09 07:47:45 +00:00)


In [ ]:
train_y = train_df['is_sarcastic']
test_y = test_df['is_sarcastic']

train_x = train_df['headline']
test_x = test_df['headline']
train_x.shape,train_y.shape,test_x.shape,test_y.shape

((22702,), (22702,), (4007,), (4007,))

time: 6.08 ms (started: 2021-08-09 07:47:45 +00:00)


In [ ]:
# BERT Embeddings

def transformer_embedding(name,inp,model_name):

    model = model_name.from_pretrained(name)
    tokenizer = AutoTokenizer.from_pretrained(name)
    pipe = pipeline('feature-extraction', model=model, 
                tokenizer=tokenizer)
    features = pipe(inp)
    features = np.squeeze(features)
    return features

time: 5.41 ms (started: 2021-08-09 07:47:46 +00:00)


In [ ]:
'''
Embedding trainset

'''
train_x_list = train_x.tolist()
train_y_list = train_y.to_list()

time: 2.7 ms (started: 2021-08-09 07:48:37 +00:00)


In [ ]:
len(train_x_list)

22702

time: 4.07 ms (started: 2021-08-09 07:48:38 +00:00)


In [ ]:
len(train_y_list)

22702

time: 11.5 ms (started: 2021-08-09 07:48:39 +00:00)


## Train 1

In [ ]:
# Embeddings trainset
embedding_features=transformer_embedding('bert-base-uncased',train_x_list[0:4000],TFBertModel)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.



time: 5min 36s (started: 2021-08-09 07:51:12 +00:00)


In [ ]:
embedding_features.shape

(4000, 37, 768)

time: 7.08 ms (started: 2021-08-09 07:56:49 +00:00)


In [ ]:
train_bert_x = embedding_features[:,0,:]

time: 1.76 ms (started: 2021-08-09 07:56:49 +00:00)


In [ ]:
train_bert_x.shape

(4000, 768)

time: 3.76 ms (started: 2021-08-09 07:56:49 +00:00)


In [ ]:
train_bert_x_df = pd.DataFrame(train_bert_x)

time: 4.23 ms (started: 2021-08-09 07:56:49 +00:00)


In [ ]:
train_bert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.190443,0.249115,0.230637,0.046937,-0.425034,-0.272725,0.469147,0.277338,0.008943,-0.050652,-0.017815,-0.104479,0.073786,-0.012094,-0.111905,0.241476,-0.263029,0.073332,0.344830,-0.280562,0.107353,0.028069,-0.080546,-0.326237,0.038039,-0.017133,-0.000988,-0.234635,0.077146,0.459631,0.171129,0.152891,-0.028653,-0.207002,-0.061835,-0.133595,0.480229,0.254105,0.224409,0.312456,...,-0.037141,-0.202688,-0.070961,0.208641,-0.240333,-0.106162,-0.066628,0.101141,0.057052,0.007349,0.063804,0.365955,0.294087,-0.070599,0.379931,0.069376,0.336008,-0.118725,-0.082539,-0.081337,-0.055999,-0.161743,-0.017171,0.261253,-7.710991,-0.120480,-0.371195,-0.246646,0.074852,-0.224048,-0.314523,-0.169071,-0.029449,0.115963,-0.030112,0.272454,0.080770,-0.199147,0.484466,0.361765
1,-0.492397,-0.027411,-0.332466,0.020443,-0.610060,-0.148527,0.654926,0.308514,-0.231310,-0.069617,-0.002004,-0.280929,0.227047,0.351792,-0.081066,-0.243597,-0.295800,0.535180,0.217020,-0.006277,-0.016861,-0.292270,0.045566,-0.195326,0.149736,0.058718,0.298146,-0.352757,-0.024628,0.090480,0.199756,0.139113,-0.203298,-0.230459,0.253481,-0.294298,0.042615,0.352720,0.151282,0.150157,...,0.136102,-0.153747,-0.003750,0.462145,-0.426525,-0.141466,-0.360657,0.167633,0.092457,-0.143379,-0.327042,0.332397,0.360107,0.058052,0.266458,0.623569,0.143433,0.375017,0.051152,0.087787,-0.061374,-0.373077,0.289576,0.027173,-6.620026,-0.146909,-0.239583,0.060715,0.025357,-0.414859,-0.141133,0.228207,-0.018855,0.251574,0.218195,0.030464,-0.055820,-0.330648,0.540078,0.175039
2,-0.142799,-0.049289,0.228374,0.020959,-0.200608,-0.045387,0.334416,0.423211,-0.062562,-0.085432,-0.052321,-0.324418,0.136056,0.254049,-0.144795,-0.068938,-0.091523,0.078198,0.236178,-0.176171,-0.107625,-0.274301,-0.115203,-0.137790,0.255695,-0.239128,-0.193295,0.059538,0.189012,0.060158,-0.151148,0.134901,0.110153,0.030933,0.022396,0.065900,-0.086931,-0.064722,-0.112232,-0.054907,...,-0.225551,0.098249,-0.072251,0.135678,-0.098235,-0.133579,-0.321829,-0.142423,0.163555,-0.035497,0.048289,0.128056,0.057226,0.095806,0.247166,0.156255,0.068330,0.002394,0.214267,0.035531,-0.193269,-0.386303,0.063641,-0.027111,-8.786839,-0.177657,-0.019461,-0.183167,-0.132528,-0.192282,0.074134,-0.213986,-0.067644,0.216824,0.095072,-0.164247,-0.103283,-0.315215,0.064099,0.204661
3,-0.868000,-0.101078,-0.575666,0.459852,-0.419928,0.016022,0.334006,0.135863,-0.458442,-0.373681,0.098813,-0.304566,-0.724307,0.561535,-0.098482,0.016669,-0.008255,0.168122,-0.026358,-0.096314,0.316348,-0.272942,0.199998,-0.181488,-0.334703,-0.286956,-0.097895,-0.104310,-0.358083,0.068524,-0.019952,-0.093822,-0.243518,-0.114497,0.153138,-0.228161,0.460579,-0.323959,0.125136,0.356893,...,0.309591,0.274705,-0.066187,0.446192,0.004475,-0.288398,-0.041143,0.137002,0.599168,0.097013,-0.010967,-0.321441,0.173009,-0.415096,0.366950,0.486005,-0.390781,0.732672,0.274839,0.110842,-0.084790,-0.715592,0.140092,-0.168720,-5.533344,-0.255496,-0.490340,-0.269528,0.256371,-0.403477,-0.155346,-0.029823,0.271217,0.259728,0.422689,-0.150633,-0.098891,-0.350663,0.440300,0.144549
4,-0.766649,-0.008375,-0.152243,-0.141487,-0.433392,0.113954,0.108358,0.462332,-0.292498,-0.144439,-0.006162,0.005111,-0.093738,0.347403,-0.038388,-0.089119,-0.247187,0.433039,0.025288,-0.218991,0.147774,-0.068135,0.019007,-0.186180,0.399780,0.054598,-0.008111,-0.578480,0.104578,0.158246,0.070361,0.438630,-0.099483,0.172906,-0.088207,-0.365570,0.203475,0.052855,0.113435,0.233232,...,0.623833,-0.399734,0.280004,0.302165,-0.213661,-0.130853,-0.680508,0.219384,-0.330596,0.184579,-0.553960,0.838544,0.014515,-0.066736,0.215119,0.592264,0.087756,-0.129169,0.353065,0.063154,-0.208683,-0.264319,0.578415,0.125885,-6.732278,-0.223226,-0.731024,-0.099794,

time: 149 ms (started: 2021-08-09 07:56:49 +00:00)


In [ ]:
train_bert_x_df['is_sarcastic'] = train_y_list[0:4000]

time: 5.73 ms (started: 2021-08-09 07:56:49 +00:00)


In [ ]:
train_bert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,is_sarcastic
0,-0.190443,0.249115,0.230637,0.046937,-0.425034,-0.272725,0.469147,0.277338,0.008943,-0.050652,-0.017815,-0.104479,0.073786,-0.012094,-0.111905,0.241476,-0.263029,0.073332,0.344830,-0.280562,0.107353,0.028069,-0.080546,-0.326237,0.038039,-0.017133,-0.000988,-0.234635,0.077146,0.459631,0.171129,0.152891,-0.028653,-0.207002,-0.061835,-0.133595,0.480229,0.254105,0.224409,0.312456,...,-0.202688,-0.070961,0.208641,-0.240333,-0.106162,-0.066628,0.101141,0.057052,0.007349,0.063804,0.365955,0.294087,-0.070599,0.379931,0.069376,0.336008,-0.118725,-0.082539,-0.081337,-0.055999,-0.161743,-0.017171,0.261253,-7.710991,-0.120480,-0.371195,-0.246646,0.074852,-0.224048,-0.314523,-0.169071,-0.029449,0.115963,-0.030112,0.272454,0.080770,-0.199147,0.484466,0.361765,0
1,-0.492397,-0.027411,-0.332466,0.020443,-0.610060,-0.148527,0.654926,0.308514,-0.231310,-0.069617,-0.002004,-0.280929,0.227047,0.351792,-0.081066,-0.243597,-0.295800,0.535180,0.217020,-0.006277,-0.016861,-0.292270,0.045566,-0.195326,0.149736,0.058718,0.298146,-0.352757,-0.024628,0.090480,0.199756,0.139113,-0.203298,-0.230459,0.253481,-0.294298,0.042615,0.352720,0.151282,0.150157,...,-0.153747,-0.003750,0.462145,-0.426525,-0.141466,-0.360657,0.167633,0.092457,-0.143379,-0.327042,0.332397,0.360107,0.058052,0.266458,0.623569,0.143433,0.375017,0.051152,0.087787,-0.061374,-0.373077,0.289576,0.027173,-6.620026,-0.146909,-0.239583,0.060715,0.025357,-0.414859,-0.141133,0.228207,-0.018855,0.251574,0.218195,0.030464,-0.055820,-0.330648,0.540078,0.175039,1
2,-0.142799,-0.049289,0.228374,0.020959,-0.200608,-0.045387,0.334416,0.423211,-0.062562,-0.085432,-0.052321,-0.324418,0.136056,0.254049,-0.144795,-0.068938,-0.091523,0.078198,0.236178,-0.176171,-0.107625,-0.274301,-0.115203,-0.137790,0.255695,-0.239128,-0.193295,0.059538,0.189012,0.060158,-0.151148,0.134901,0.110153,0.030933,0.022396,0.065900,-0.086931,-0.064722,-0.112232,-0.054907,...,0.098249,-0.072251,0.135678,-0.098235,-0.133579,-0.321829,-0.142423,0.163555,-0.035497,0.048289,0.128056,0.057226,0.095806,0.247166,0.156255,0.068330,0.002394,0.214267,0.035531,-0.193269,-0.386303,0.063641,-0.027111,-8.786839,-0.177657,-0.019461,-0.183167,-0.132528,-0.192282,0.074134,-0.213986,-0.067644,0.216824,0.095072,-0.164247,-0.103283,-0.315215,0.064099,0.204661,0
3,-0.868000,-0.101078,-0.575666,0.459852,-0.419928,0.016022,0.334006,0.135863,-0.458442,-0.373681,0.098813,-0.304566,-0.724307,0.561535,-0.098482,0.016669,-0.008255,0.168122,-0.026358,-0.096314,0.316348,-0.272942,0.199998,-0.181488,-0.334703,-0.286956,-0.097895,-0.104310,-0.358083,0.068524,-0.019952,-0.093822,-0.243518,-0.114497,0.153138,-0.228161,0.460579,-0.323959,0.125136,0.356893,...,0.274705,-0.066187,0.446192,0.004475,-0.288398,-0.041143,0.137002,0.599168,0.097013,-0.010967,-0.321441,0.173009,-0.415096,0.366950,0.486005,-0.390781,0.732672,0.274839,0.110842,-0.084790,-0.715592,0.140092,-0.168720,-5.533344,-0.255496,-0.490340,-0.269528,0.256371,-0.403477,-0.155346,-0.029823,0.271217,0.259728,0.422689,-0.150633,-0.098891,-0.350663,0.440300,0.144549,1
4,-0.766649,-0.008375,-0.152243,-0.141487,-0.433392,0.113954,0.108358,0.462332,-0.292498,-0.144439,-0.006162,0.005111,-0.093738,0.347403,-0.038388,-0.089119,-0.247187,0.433039,0.025288,-0.218991,0.147774,-0.068135,0.019007,-0.186180,0.399780,0.054598,-0.008111,-0.578480,0.104578,0.158246,0.070361,0.438630,-0.099483,0.172906,-0.088207,-0.365570,0.203475,0.052855,0.113435,0.233232,...,-0.399734,0.280004,0.302165,-0.213661,-0.130853,-0.680508,0.219384,-0.330596,0.184579,-0.553960,0.838544,0.014515,-0.066736,0.215119,0.592264,0.087756,-0.129169,0.353065,0.063154,-0.208683,-0.264319,0.578415,0.125885,-6.732278,-0.223226,-0.731024,-0.099794,0.313067,-0.231150,0.110340,0.

time: 140 ms (started: 2021-08-09 07:56:49 +00:00)


In [ ]:
train_bert_x_df.to_csv('datasets/BERT/train_bert_0_4000.csv', encoding='utf-8', index=False)

time: 5.85 s (started: 2021-08-09 07:56:49 +00:00)


## Train 2

In [ ]:
# Embeddings trainset
embedding_features=transformer_embedding('bert-base-uncased',train_x_list[4000:8000],TFBertModel)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


time: 3min 32s (started: 2021-08-09 08:05:46 +00:00)


In [ ]:
embedding_features.shape

(4000, 24, 768)

time: 5.29 ms (started: 2021-08-09 08:09:19 +00:00)


In [ ]:
train_bert_x = embedding_features[:,0,:]

time: 732 µs (started: 2021-08-09 08:09:19 +00:00)


In [ ]:
train_bert_x.shape

(4000, 768)

time: 5.48 ms (started: 2021-08-09 08:09:19 +00:00)


In [ ]:
train_bert_x_df = pd.DataFrame(train_bert_x)

time: 2.69 ms (started: 2021-08-09 08:09:19 +00:00)


In [ ]:
train_bert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.487389,0.095127,-0.388939,0.107141,-0.235933,-0.011764,-0.131868,-0.175145,-0.693373,-0.058755,0.166755,0.283896,-0.061933,0.107337,-0.009057,0.154273,-0.182717,0.584371,-0.043375,-0.139766,0.406772,-0.076594,-0.045740,-0.216778,-0.116738,0.022751,0.076591,-0.149539,-0.191742,0.085162,-0.048894,-0.118353,-0.080302,-0.220546,0.216686,-0.308540,0.358809,-0.560071,0.193967,0.155643,...,0.225503,-0.119470,0.141567,0.469498,0.011914,-0.059922,-0.381806,-0.020283,0.483346,-0.271493,-0.477155,0.041585,0.292870,-0.150641,0.652130,0.277654,0.042758,0.318313,0.055771,0.054241,-0.515492,-0.178017,0.453364,-0.068238,-7.404553,-0.093516,-0.291954,-0.252758,0.424815,-0.574976,0.017353,-0.059617,0.440040,0.169022,0.375776,0.154379,-0.157376,-0.203853,0.046877,0.527030
1,-0.243699,0.218615,0.130719,0.153181,0.009295,0.156285,0.364808,0.051124,-0.606116,-0.262508,-0.129741,0.145469,-0.386014,0.537300,-0.478060,0.508210,-0.035185,0.232895,0.378822,-0.150314,0.000896,-0.291851,0.198582,-0.026515,0.064741,-0.128586,0.280700,0.297320,0.047498,0.451017,-0.146428,0.601446,0.087791,0.166747,-0.065281,-0.470455,0.532152,-0.203795,0.161586,0.107459,...,0.338009,-0.536540,0.038433,-0.121126,-0.472544,-0.204524,-0.247919,-0.166131,0.169231,-0.058636,-0.071614,0.117580,0.162087,-0.367189,-0.131542,0.337281,0.490238,0.080729,0.326350,-0.050184,0.036772,-0.514038,0.300315,0.407028,-6.776921,0.026832,-0.212880,-0.221345,0.078229,0.269207,0.153125,-0.046667,-0.234734,0.051659,0.534897,0.055663,-0.266331,-0.486827,0.259728,-0.025763
2,0.533739,0.197596,-0.057717,0.178245,-0.127621,-0.631013,0.082628,0.992981,0.000993,-0.027897,0.096988,-0.732429,0.157920,0.446312,0.427846,0.022019,-0.177134,0.327320,0.520699,-0.016199,0.194999,-0.244581,-0.347451,-0.265668,0.445530,0.137249,-0.305874,-0.210639,0.371100,-0.080810,-0.296369,0.630132,-0.106827,-0.643585,0.186449,-0.534760,0.957165,-0.069659,0.268184,0.373121,...,0.230433,-0.230229,0.229892,0.289960,-0.329406,0.274093,-0.074457,-0.309806,0.076764,-0.358985,-0.297678,0.721903,0.207124,-0.246827,0.198331,0.630396,0.010594,0.550572,0.482141,-0.365010,-0.051911,-0.510703,0.343657,0.302014,-6.798889,-0.353234,-0.206889,0.201413,-0.414042,-0.945528,-0.679540,-0.133130,-0.063991,0.007246,0.129607,0.236357,-0.063365,-0.445710,0.278400,-0.038013
3,-0.738714,-0.258002,-0.361993,0.081992,-0.217324,0.088843,0.072691,0.213305,-0.454709,0.030724,0.386997,-0.117693,0.424963,0.359086,0.139084,-0.108744,-0.152032,0.541668,0.416226,-0.128654,0.165798,-0.180274,0.018276,-0.460835,-0.004342,0.092546,-0.079306,-0.173065,0.158884,-0.372083,0.081772,-0.083944,-0.115084,-0.138879,0.447934,-0.253607,0.600700,-0.147483,0.288802,0.060204,...,0.450874,0.325883,0.116532,0.616460,-0.151182,-0.333299,-0.063721,-0.080932,0.182751,-0.174573,-0.078391,-0.150702,0.386323,-0.093971,-0.018548,0.280875,0.104035,0.685963,0.562132,-0.063719,-0.025335,-0.079113,0.273403,0.072121,-7.442991,-0.194182,-0.055328,-0.381684,0.163828,-0.190405,0.086067,-0.034855,0.165572,0.125614,0.428932,-0.409594,0.178506,-0.061490,0.106860,0.288059
4,-0.699753,-0.206459,-0.088463,-0.237407,0.491338,0.173599,0.264555,0.965056,-0.394819,0.156832,0.005545,-0.815713,0.235518,0.531930,-0.162735,-0.363914,-0.341819,0.690730,0.090034,0.062376,0.452852,-0.483394,-0.255411,-0.031346,-0.228638,-0.073071,-0.368432,-0.033434,-0.086319,-0.009330,0.012583,0.138497,0.443327,0.038111,0.224063,-0.907948,0.889804,0.012932,-0.292625,0.154929,...,0.231882,-0.089421,0.210069,0.336162,-0.456915,-0.162804,-0.696859,0.075153,-0.244094,-0.213273,-0.632978,0.448704,0.100573,-0.109807,0.391334,0.328520,0.264812,0.331932,0.822198,0.068291,-0.071493,-0.384399,0.402081,-0.059041,-6.893677,-0.355561,-0.205284,-0.521891,0.205217,-0

time: 181 ms (started: 2021-08-09 08:09:19 +00:00)


In [ ]:
train_bert_x_df['is_sarcastic'] = train_y_list[4000:8000]

time: 6.7 ms (started: 2021-08-09 08:09:19 +00:00)


In [ ]:
train_bert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,is_sarcastic
0,-0.487389,0.095127,-0.388939,0.107141,-0.235933,-0.011764,-0.131868,-0.175145,-0.693373,-0.058755,0.166755,0.283896,-0.061933,0.107337,-0.009057,0.154273,-0.182717,0.584371,-0.043375,-0.139766,0.406772,-0.076594,-0.045740,-0.216778,-0.116738,0.022751,0.076591,-0.149539,-0.191742,0.085162,-0.048894,-0.118353,-0.080302,-0.220546,0.216686,-0.308540,0.358809,-0.560071,0.193967,0.155643,...,-0.119470,0.141567,0.469498,0.011914,-0.059922,-0.381806,-0.020283,0.483346,-0.271493,-0.477155,0.041585,0.292870,-0.150641,0.652130,0.277654,0.042758,0.318313,0.055771,0.054241,-0.515492,-0.178017,0.453364,-0.068238,-7.404553,-0.093516,-0.291954,-0.252758,0.424815,-0.574976,0.017353,-0.059617,0.440040,0.169022,0.375776,0.154379,-0.157376,-0.203853,0.046877,0.527030,1
1,-0.243699,0.218615,0.130719,0.153181,0.009295,0.156285,0.364808,0.051124,-0.606116,-0.262508,-0.129741,0.145469,-0.386014,0.537300,-0.478060,0.508210,-0.035185,0.232895,0.378822,-0.150314,0.000896,-0.291851,0.198582,-0.026515,0.064741,-0.128586,0.280700,0.297320,0.047498,0.451017,-0.146428,0.601446,0.087791,0.166747,-0.065281,-0.470455,0.532152,-0.203795,0.161586,0.107459,...,-0.536540,0.038433,-0.121126,-0.472544,-0.204524,-0.247919,-0.166131,0.169231,-0.058636,-0.071614,0.117580,0.162087,-0.367189,-0.131542,0.337281,0.490238,0.080729,0.326350,-0.050184,0.036772,-0.514038,0.300315,0.407028,-6.776921,0.026832,-0.212880,-0.221345,0.078229,0.269207,0.153125,-0.046667,-0.234734,0.051659,0.534897,0.055663,-0.266331,-0.486827,0.259728,-0.025763,1
2,0.533739,0.197596,-0.057717,0.178245,-0.127621,-0.631013,0.082628,0.992981,0.000993,-0.027897,0.096988,-0.732429,0.157920,0.446312,0.427846,0.022019,-0.177134,0.327320,0.520699,-0.016199,0.194999,-0.244581,-0.347451,-0.265668,0.445530,0.137249,-0.305874,-0.210639,0.371100,-0.080810,-0.296369,0.630132,-0.106827,-0.643585,0.186449,-0.534760,0.957165,-0.069659,0.268184,0.373121,...,-0.230229,0.229892,0.289960,-0.329406,0.274093,-0.074457,-0.309806,0.076764,-0.358985,-0.297678,0.721903,0.207124,-0.246827,0.198331,0.630396,0.010594,0.550572,0.482141,-0.365010,-0.051911,-0.510703,0.343657,0.302014,-6.798889,-0.353234,-0.206889,0.201413,-0.414042,-0.945528,-0.679540,-0.133130,-0.063991,0.007246,0.129607,0.236357,-0.063365,-0.445710,0.278400,-0.038013,1
3,-0.738714,-0.258002,-0.361993,0.081992,-0.217324,0.088843,0.072691,0.213305,-0.454709,0.030724,0.386997,-0.117693,0.424963,0.359086,0.139084,-0.108744,-0.152032,0.541668,0.416226,-0.128654,0.165798,-0.180274,0.018276,-0.460835,-0.004342,0.092546,-0.079306,-0.173065,0.158884,-0.372083,0.081772,-0.083944,-0.115084,-0.138879,0.447934,-0.253607,0.600700,-0.147483,0.288802,0.060204,...,0.325883,0.116532,0.616460,-0.151182,-0.333299,-0.063721,-0.080932,0.182751,-0.174573,-0.078391,-0.150702,0.386323,-0.093971,-0.018548,0.280875,0.104035,0.685963,0.562132,-0.063719,-0.025335,-0.079113,0.273403,0.072121,-7.442991,-0.194182,-0.055328,-0.381684,0.163828,-0.190405,0.086067,-0.034855,0.165572,0.125614,0.428932,-0.409594,0.178506,-0.061490,0.106860,0.288059,0
4,-0.699753,-0.206459,-0.088463,-0.237407,0.491338,0.173599,0.264555,0.965056,-0.394819,0.156832,0.005545,-0.815713,0.235518,0.531930,-0.162735,-0.363914,-0.341819,0.690730,0.090034,0.062376,0.452852,-0.483394,-0.255411,-0.031346,-0.228638,-0.073071,-0.368432,-0.033434,-0.086319,-0.009330,0.012583,0.138497,0.443327,0.038111,0.224063,-0.907948,0.889804,0.012932,-0.292625,0.154929,...,-0.089421,0.210069,0.336162,-0.456915,-0.162804,-0.696859,0.075153,-0.244094,-0.213273,-0.632978,0.448704,0.100573,-0.109807,0.391334,0.328520,0.264812,0.331932,0.822198,0.068291,-0.071493,-0.384399,0.402081,-0.059041,-6.893677,-0.355561,-0.205284,-0.521891,0.205217,-0.452547,0.301594,-0.024069,0

time: 157 ms (started: 2021-08-09 08:09:19 +00:00)


In [ ]:
train_bert_x_df.to_csv('datasets/BERT/train_bert_4000_8000.csv', encoding='utf-8', index=False)

time: 5.62 s (started: 2021-08-09 08:09:19 +00:00)


## Train 3

In [ ]:
# Embeddings trainset
embedding_features=transformer_embedding('bert-base-uncased',train_x_list[8000:12000],TFBertModel)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


time: 4min 41s (started: 2021-08-09 08:09:25 +00:00)


In [ ]:
embedding_features.shape

(4000, 32, 768)

time: 4.3 ms (started: 2021-08-09 08:14:07 +00:00)


In [ ]:
train_bert_x = embedding_features[:,0,:]

time: 1.45 ms (started: 2021-08-09 08:14:07 +00:00)


In [ ]:
train_bert_x.shape

(4000, 768)

time: 4.14 ms (started: 2021-08-09 08:14:07 +00:00)


In [ ]:
train_bert_x_df = pd.DataFrame(train_bert_x)

time: 1.94 ms (started: 2021-08-09 08:14:07 +00:00)


In [ ]:
train_bert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.650271,-0.231898,-0.445162,0.047825,-0.111458,-0.060220,0.058105,0.045895,-0.381737,-0.146507,0.122267,-0.018041,-0.106601,0.352808,0.138276,0.433364,-0.165200,0.449233,0.641363,0.078765,0.298746,-0.395811,-0.030710,-0.112541,-0.106965,-0.204872,-0.192736,-0.161541,0.114541,-0.397492,0.257550,-0.037695,-0.268712,0.053521,0.432437,0.128364,0.465252,-0.331125,0.404748,0.011944,...,0.145475,0.076045,0.232738,0.044699,0.020780,-0.720267,-0.215333,0.071835,0.150257,0.151201,0.018798,-0.260509,0.389347,-0.042694,0.233185,-0.051828,-0.048123,0.972653,0.335486,-0.086263,-0.256580,-0.326218,0.421794,-0.180406,-7.798815,-0.432182,-0.450792,-0.483669,0.404439,-0.387212,-0.187477,-0.001508,0.344228,0.013506,0.183584,-0.174781,0.035288,-0.186468,0.100707,0.258762
1,-0.419367,0.051144,0.133037,-0.135772,-0.119535,0.004289,0.658042,0.040983,-0.088435,-0.200679,0.195515,-0.341417,0.379074,0.539074,0.057817,0.124792,-0.235000,0.464491,0.308152,-0.234392,0.093204,-1.022587,0.117334,0.137988,-0.196521,0.147516,-0.129201,-0.253440,0.205979,-0.223059,0.172503,0.461478,-0.255091,-0.659285,0.514447,-0.125070,0.472968,-0.015435,-0.151514,0.607219,...,-0.100787,0.133898,0.332320,0.206160,-0.629925,-0.133578,-0.209464,-0.207130,0.114218,0.095831,-0.276807,0.158048,0.131288,-0.140163,0.451076,0.419374,0.064559,0.341833,0.470444,0.199090,0.066968,-0.360959,0.243176,0.065052,-7.640139,-0.153539,-0.542692,-0.177927,0.021276,-0.709461,-0.138773,-0.080288,-0.114673,-0.043359,0.025585,0.057984,-0.047005,-0.322405,0.111364,-0.108244
2,-0.027137,0.103812,0.199558,0.034872,-0.193866,-0.221959,0.133741,0.272089,-0.102785,-0.049325,-0.118550,0.068982,-0.177217,-0.043239,-0.195192,-0.122647,-0.214434,0.166470,0.228612,-0.182249,-0.100772,-0.302803,0.049543,-0.204825,0.189393,-0.209079,0.155444,-0.105027,0.199002,0.103510,0.025297,0.105586,-0.194625,-0.045283,0.038562,-0.075518,0.096192,-0.076237,0.150885,0.304914,...,0.100427,0.007247,-0.145975,0.201689,-0.055544,0.180978,0.176164,-0.136341,0.057113,0.101581,-0.179307,0.110910,0.223589,0.257770,0.180307,0.200497,0.119835,0.304905,0.134923,-0.316784,0.023814,-0.246462,0.054526,0.179176,-8.851394,-0.050114,-0.198379,0.072300,-0.022923,-0.063611,0.011961,-0.063991,0.154878,0.066233,-0.134532,-0.043254,0.104489,-0.271953,0.087871,0.194466
3,-0.461840,-0.149302,-0.083925,-0.167146,-0.190923,-0.138687,0.408144,0.457323,-0.408111,-0.330751,0.197319,-0.099923,-0.022305,0.784556,-0.086001,0.086252,-0.458368,0.617207,0.049929,-0.275481,0.085222,-0.363658,0.133680,-0.299042,-0.078516,-0.088123,-0.053543,-0.061028,0.170817,-0.023958,-0.098510,0.191158,-0.186328,-0.172157,0.198459,-0.499625,0.178311,0.138547,0.133187,0.216086,...,0.300024,-0.280284,-0.011765,0.255896,-0.379431,-0.275716,-0.280530,0.223602,0.017748,-0.163680,-0.263014,0.292431,0.226619,0.233966,-0.037403,0.468846,0.124868,-0.000714,0.277941,0.195624,-0.266100,-0.045713,0.178405,0.338259,-7.915963,-0.266364,-0.228246,0.082587,0.128716,-0.401326,-0.128083,-0.042603,0.291934,0.248509,0.272396,-0.516142,-0.166622,-0.169794,0.381066,0.113475
4,-0.178150,0.525380,-0.003894,-0.041336,-0.060241,-0.535904,0.490912,0.659629,-0.560164,-0.086829,0.060971,-0.076466,-0.269261,0.258403,0.048972,0.421136,-0.339171,0.253070,0.511215,0.036582,0.217295,-0.577459,-0.105650,-0.169917,-0.023737,0.183326,0.290314,0.286262,-0.222892,-0.211037,0.381935,-0.171507,0.189525,0.102173,0.337842,-0.394954,0.022735,-0.024716,0.158278,-0.114007,...,0.016782,-0.368597,0.056755,0.527477,-0.515471,0.131779,0.226932,-0.039041,-0.026320,-0.053261,-0.371536,-0.179529,0.001105,0.086092,-0.247508,-0.083602,0.367374,0.508129,0.073766,-0.030962,-0.432133,0.300916,0.460552,0.351661,-7.713204,-0.103524,-0.384959,0.244186,-0.224172,0

time: 117 ms (started: 2021-08-09 08:14:07 +00:00)


In [ ]:
train_bert_x_df['is_sarcastic'] = train_y_list[8000:12000]

time: 5.55 ms (started: 2021-08-09 08:14:07 +00:00)


In [ ]:
train_bert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,is_sarcastic
0,-0.650271,-0.231898,-0.445162,0.047825,-0.111458,-0.060220,0.058105,0.045895,-0.381737,-0.146507,0.122267,-0.018041,-0.106601,0.352808,0.138276,0.433364,-0.165200,0.449233,0.641363,0.078765,0.298746,-0.395811,-0.030710,-0.112541,-0.106965,-0.204872,-0.192736,-0.161541,0.114541,-0.397492,0.257550,-0.037695,-0.268712,0.053521,0.432437,0.128364,0.465252,-0.331125,0.404748,0.011944,...,0.076045,0.232738,0.044699,0.020780,-0.720267,-0.215333,0.071835,0.150257,0.151201,0.018798,-0.260509,0.389347,-0.042694,0.233185,-0.051828,-0.048123,0.972653,0.335486,-0.086263,-0.256580,-0.326218,0.421794,-0.180406,-7.798815,-0.432182,-0.450792,-0.483669,0.404439,-0.387212,-0.187477,-0.001508,0.344228,0.013506,0.183584,-0.174781,0.035288,-0.186468,0.100707,0.258762,0
1,-0.419367,0.051144,0.133037,-0.135772,-0.119535,0.004289,0.658042,0.040983,-0.088435,-0.200679,0.195515,-0.341417,0.379074,0.539074,0.057817,0.124792,-0.235000,0.464491,0.308152,-0.234392,0.093204,-1.022587,0.117334,0.137988,-0.196521,0.147516,-0.129201,-0.253440,0.205979,-0.223059,0.172503,0.461478,-0.255091,-0.659285,0.514447,-0.125070,0.472968,-0.015435,-0.151514,0.607219,...,0.133898,0.332320,0.206160,-0.629925,-0.133578,-0.209464,-0.207130,0.114218,0.095831,-0.276807,0.158048,0.131288,-0.140163,0.451076,0.419374,0.064559,0.341833,0.470444,0.199090,0.066968,-0.360959,0.243176,0.065052,-7.640139,-0.153539,-0.542692,-0.177927,0.021276,-0.709461,-0.138773,-0.080288,-0.114673,-0.043359,0.025585,0.057984,-0.047005,-0.322405,0.111364,-0.108244,1
2,-0.027137,0.103812,0.199558,0.034872,-0.193866,-0.221959,0.133741,0.272089,-0.102785,-0.049325,-0.118550,0.068982,-0.177217,-0.043239,-0.195192,-0.122647,-0.214434,0.166470,0.228612,-0.182249,-0.100772,-0.302803,0.049543,-0.204825,0.189393,-0.209079,0.155444,-0.105027,0.199002,0.103510,0.025297,0.105586,-0.194625,-0.045283,0.038562,-0.075518,0.096192,-0.076237,0.150885,0.304914,...,0.007247,-0.145975,0.201689,-0.055544,0.180978,0.176164,-0.136341,0.057113,0.101581,-0.179307,0.110910,0.223589,0.257770,0.180307,0.200497,0.119835,0.304905,0.134923,-0.316784,0.023814,-0.246462,0.054526,0.179176,-8.851394,-0.050114,-0.198379,0.072300,-0.022923,-0.063611,0.011961,-0.063991,0.154878,0.066233,-0.134532,-0.043254,0.104489,-0.271953,0.087871,0.194466,0
3,-0.461840,-0.149302,-0.083925,-0.167146,-0.190923,-0.138687,0.408144,0.457323,-0.408111,-0.330751,0.197319,-0.099923,-0.022305,0.784556,-0.086001,0.086252,-0.458368,0.617207,0.049929,-0.275481,0.085222,-0.363658,0.133680,-0.299042,-0.078516,-0.088123,-0.053543,-0.061028,0.170817,-0.023958,-0.098510,0.191158,-0.186328,-0.172157,0.198459,-0.499625,0.178311,0.138547,0.133187,0.216086,...,-0.280284,-0.011765,0.255896,-0.379431,-0.275716,-0.280530,0.223602,0.017748,-0.163680,-0.263014,0.292431,0.226619,0.233966,-0.037403,0.468846,0.124868,-0.000714,0.277941,0.195624,-0.266100,-0.045713,0.178405,0.338259,-7.915963,-0.266364,-0.228246,0.082587,0.128716,-0.401326,-0.128083,-0.042603,0.291934,0.248509,0.272396,-0.516142,-0.166622,-0.169794,0.381066,0.113475,1
4,-0.178150,0.525380,-0.003894,-0.041336,-0.060241,-0.535904,0.490912,0.659629,-0.560164,-0.086829,0.060971,-0.076466,-0.269261,0.258403,0.048972,0.421136,-0.339171,0.253070,0.511215,0.036582,0.217295,-0.577459,-0.105650,-0.169917,-0.023737,0.183326,0.290314,0.286262,-0.222892,-0.211037,0.381935,-0.171507,0.189525,0.102173,0.337842,-0.394954,0.022735,-0.024716,0.158278,-0.114007,...,-0.368597,0.056755,0.527477,-0.515471,0.131779,0.226932,-0.039041,-0.026320,-0.053261,-0.371536,-0.179529,0.001105,0.086092,-0.247508,-0.083602,0.367374,0.508129,0.073766,-0.030962,-0.432133,0.300916,0.460552,0.351661,-7.713204,-0.103524,-0.384959,0.244186,-0.224172,0.037925,0.452117,0.032464,-0.

time: 126 ms (started: 2021-08-09 08:14:07 +00:00)


In [ ]:
train_bert_x_df.to_csv('datasets/BERT/train_bert_8000_12000.csv', encoding='utf-8', index=False)

time: 5.7 s (started: 2021-08-09 08:14:07 +00:00)


## Train 4

In [ ]:
# Embeddings trainset
embedding_features=transformer_embedding('bert-base-uncased',train_x_list[12000:16000],TFBertModel)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


time: 5min 48s (started: 2021-08-09 08:14:13 +00:00)


In [ ]:
embedding_features.shape

(4000, 35, 768)

time: 65.5 ms (started: 2021-08-09 08:20:01 +00:00)


In [ ]:
train_bert_x = embedding_features[:,0,:]

time: 1.41 ms (started: 2021-08-09 08:20:01 +00:00)


In [ ]:
train_bert_x.shape

(4000, 768)

time: 8.59 ms (started: 2021-08-09 08:20:01 +00:00)


In [ ]:
train_bert_x_df = pd.DataFrame(train_bert_x)

time: 2.33 ms (started: 2021-08-09 08:20:02 +00:00)


In [ ]:
train_bert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.244856,0.004846,0.159973,0.136028,-0.068536,-0.059607,0.511311,0.252292,-0.295975,-0.429122,0.108375,0.038616,0.100969,0.214033,0.203709,0.335224,-0.306926,0.406187,0.297326,-0.310882,0.196862,-0.320097,0.010244,-0.024915,-0.151354,-0.031629,-0.087448,0.080035,0.023525,0.056014,0.084492,-0.024680,-0.068019,0.275453,-0.057617,-0.081929,0.375636,-0.143589,0.255590,0.197829,...,0.171614,0.092300,0.159666,0.305979,0.002926,-0.098924,-0.214151,-0.259414,0.320579,0.056956,0.187123,-0.107932,0.088997,-0.086034,-0.098124,0.235079,0.078988,0.379652,0.328681,0.246010,-0.127812,0.061818,0.298648,0.109211,-8.737410,-0.149782,-0.118614,0.147653,0.100175,-0.356348,-0.375030,0.006106,0.090874,0.122415,0.188797,0.041254,0.054766,-0.438728,0.060710,0.241150
1,-0.656429,0.833665,-0.566433,-0.202905,-0.080435,0.062189,0.746020,0.823491,-0.875822,-0.271931,-0.320224,-0.609260,0.223638,0.790730,0.136812,-0.034605,-0.537260,0.676462,0.067374,-0.379539,0.318264,-0.339642,-0.101261,0.187508,0.337408,0.022926,0.329270,0.254838,-0.055391,-0.295581,-0.399403,0.650272,-0.027089,-0.025021,0.531900,-0.028340,0.320505,-0.177309,0.633091,0.308052,...,0.275377,-0.025350,0.007067,0.131833,-0.719120,0.180865,0.052040,0.126492,-0.105027,0.230059,-0.168765,0.156026,0.291079,0.067881,0.622324,-0.188910,0.290588,0.383875,0.041625,-0.270106,0.152467,-0.425799,0.191310,0.131993,-6.386671,-0.048486,-0.646747,0.078874,-0.115778,-0.004538,0.160212,-0.386535,-0.056896,0.243206,0.182390,0.439605,0.245678,-0.327188,0.514018,0.157723
2,-0.245419,0.159389,-0.227434,-0.113486,-0.383217,-0.060383,0.263608,0.504460,-0.068973,-0.033588,0.006996,-0.086397,0.073509,0.469897,0.161928,0.014506,-0.444120,0.414067,0.290023,-0.096196,-0.034948,-0.239246,-0.185427,-0.218308,0.142636,-0.212659,0.086860,-0.294337,0.136189,-0.008760,0.050282,0.085746,0.100282,-0.019740,0.073847,-0.148583,0.052050,-0.104771,0.146118,0.004040,...,0.127032,-0.015173,0.023375,0.275314,-0.366385,-0.015308,-0.164428,0.204506,0.012885,-0.067472,-0.205732,0.292807,0.298751,-0.101815,0.042151,0.450020,0.306265,0.235910,0.157163,0.006686,-0.312315,-0.245056,0.220224,0.211603,-8.690742,-0.417252,-0.171170,-0.214097,-0.015640,-0.291008,0.038000,-0.084091,0.064776,0.067914,0.123056,-0.089572,-0.123485,-0.285171,0.192953,0.618247
3,-0.538403,-0.262061,0.224823,0.448465,-0.193997,-0.276792,1.070282,0.364188,-0.453377,-0.205652,-0.410822,-0.203111,0.091243,0.169939,-0.023584,0.184318,-0.127477,0.434161,0.465585,-0.269782,-0.140005,-0.324288,-0.359613,-0.202230,0.461518,-0.022623,-0.323424,-0.401196,0.057757,0.001974,0.260127,-0.473412,-0.614751,-0.234061,-0.057198,-0.325422,0.104077,-0.342906,-0.179928,0.039868,...,-0.064517,-0.313913,0.058598,0.511616,-0.182956,-0.201336,-0.657111,0.556242,-0.402316,0.276648,0.192359,0.282057,-0.133989,0.048669,0.160482,-0.336564,-0.004709,0.523999,0.116275,0.043768,0.005800,-0.322875,-0.019436,0.221734,-6.429806,-0.099915,-0.621545,0.027938,0.098123,-0.956468,-0.107644,-0.106399,0.797916,0.149219,-0.155783,-0.113772,0.244595,-0.984094,0.332428,0.351421
4,-0.253055,0.145976,-0.099600,-0.244455,0.062405,-0.252726,0.106934,0.462036,-0.266056,-0.134335,-0.186626,-0.131609,-0.387705,0.174421,0.046839,-0.081192,-0.372988,0.242572,0.281523,-0.020084,-0.119921,-0.076539,-0.219199,-0.243665,0.004178,0.009428,0.129476,-0.054147,-0.087460,-0.212897,0.041128,0.048351,0.247853,-0.127717,0.000086,-0.192511,-0.018185,-0.055335,-0.041758,0.016298,...,-0.032064,-0.206814,0.156525,0.151336,-0.437203,0.083117,-0.104911,0.222071,-0.257101,-0.143557,-0.331503,0.325430,0.081955,-0.039244,0.003035,-0.253259,0.104809,0.038351,0.032706,-0.002861,-0.037736,-0.242359,0.114141,0.234049,-8.432825,-0.362574,-0.137178,0.021792,0.110004,-0.142

time: 124 ms (started: 2021-08-09 08:20:02 +00:00)


In [ ]:
train_bert_x_df['is_sarcastic'] = train_y_list[12000:16000]

time: 5.87 ms (started: 2021-08-09 08:20:02 +00:00)


In [ ]:
train_bert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,is_sarcastic
0,-0.244856,0.004846,0.159973,0.136028,-0.068536,-0.059607,0.511311,0.252292,-0.295975,-0.429122,0.108375,0.038616,0.100969,0.214033,0.203709,0.335224,-0.306926,0.406187,0.297326,-0.310882,0.196862,-0.320097,0.010244,-0.024915,-0.151354,-0.031629,-0.087448,0.080035,0.023525,0.056014,0.084492,-0.024680,-0.068019,0.275453,-0.057617,-0.081929,0.375636,-0.143589,0.255590,0.197829,...,0.092300,0.159666,0.305979,0.002926,-0.098924,-0.214151,-0.259414,0.320579,0.056956,0.187123,-0.107932,0.088997,-0.086034,-0.098124,0.235079,0.078988,0.379652,0.328681,0.246010,-0.127812,0.061818,0.298648,0.109211,-8.737410,-0.149782,-0.118614,0.147653,0.100175,-0.356348,-0.375030,0.006106,0.090874,0.122415,0.188797,0.041254,0.054766,-0.438728,0.060710,0.241150,1
1,-0.656429,0.833665,-0.566433,-0.202905,-0.080435,0.062189,0.746020,0.823491,-0.875822,-0.271931,-0.320224,-0.609260,0.223638,0.790730,0.136812,-0.034605,-0.537260,0.676462,0.067374,-0.379539,0.318264,-0.339642,-0.101261,0.187508,0.337408,0.022926,0.329270,0.254838,-0.055391,-0.295581,-0.399403,0.650272,-0.027089,-0.025021,0.531900,-0.028340,0.320505,-0.177309,0.633091,0.308052,...,-0.025350,0.007067,0.131833,-0.719120,0.180865,0.052040,0.126492,-0.105027,0.230059,-0.168765,0.156026,0.291079,0.067881,0.622324,-0.188910,0.290588,0.383875,0.041625,-0.270106,0.152467,-0.425799,0.191310,0.131993,-6.386671,-0.048486,-0.646747,0.078874,-0.115778,-0.004538,0.160212,-0.386535,-0.056896,0.243206,0.182390,0.439605,0.245678,-0.327188,0.514018,0.157723,0
2,-0.245419,0.159389,-0.227434,-0.113486,-0.383217,-0.060383,0.263608,0.504460,-0.068973,-0.033588,0.006996,-0.086397,0.073509,0.469897,0.161928,0.014506,-0.444120,0.414067,0.290023,-0.096196,-0.034948,-0.239246,-0.185427,-0.218308,0.142636,-0.212659,0.086860,-0.294337,0.136189,-0.008760,0.050282,0.085746,0.100282,-0.019740,0.073847,-0.148583,0.052050,-0.104771,0.146118,0.004040,...,-0.015173,0.023375,0.275314,-0.366385,-0.015308,-0.164428,0.204506,0.012885,-0.067472,-0.205732,0.292807,0.298751,-0.101815,0.042151,0.450020,0.306265,0.235910,0.157163,0.006686,-0.312315,-0.245056,0.220224,0.211603,-8.690742,-0.417252,-0.171170,-0.214097,-0.015640,-0.291008,0.038000,-0.084091,0.064776,0.067914,0.123056,-0.089572,-0.123485,-0.285171,0.192953,0.618247,0
3,-0.538403,-0.262061,0.224823,0.448465,-0.193997,-0.276792,1.070282,0.364188,-0.453377,-0.205652,-0.410822,-0.203111,0.091243,0.169939,-0.023584,0.184318,-0.127477,0.434161,0.465585,-0.269782,-0.140005,-0.324288,-0.359613,-0.202230,0.461518,-0.022623,-0.323424,-0.401196,0.057757,0.001974,0.260127,-0.473412,-0.614751,-0.234061,-0.057198,-0.325422,0.104077,-0.342906,-0.179928,0.039868,...,-0.313913,0.058598,0.511616,-0.182956,-0.201336,-0.657111,0.556242,-0.402316,0.276648,0.192359,0.282057,-0.133989,0.048669,0.160482,-0.336564,-0.004709,0.523999,0.116275,0.043768,0.005800,-0.322875,-0.019436,0.221734,-6.429806,-0.099915,-0.621545,0.027938,0.098123,-0.956468,-0.107644,-0.106399,0.797916,0.149219,-0.155783,-0.113772,0.244595,-0.984094,0.332428,0.351421,0
4,-0.253055,0.145976,-0.099600,-0.244455,0.062405,-0.252726,0.106934,0.462036,-0.266056,-0.134335,-0.186626,-0.131609,-0.387705,0.174421,0.046839,-0.081192,-0.372988,0.242572,0.281523,-0.020084,-0.119921,-0.076539,-0.219199,-0.243665,0.004178,0.009428,0.129476,-0.054147,-0.087460,-0.212897,0.041128,0.048351,0.247853,-0.127717,0.000086,-0.192511,-0.018185,-0.055335,-0.041758,0.016298,...,-0.206814,0.156525,0.151336,-0.437203,0.083117,-0.104911,0.222071,-0.257101,-0.143557,-0.331503,0.325430,0.081955,-0.039244,0.003035,-0.253259,0.104809,0.038351,0.032706,-0.002861,-0.037736,-0.242359,0.114141,0.234049,-8.432825,-0.362574,-0.137178,0.021792,0.110004,-0.142074,0.329733,-0.086353,0.02991

time: 138 ms (started: 2021-08-09 08:20:02 +00:00)


In [ ]:
train_bert_x_df.to_csv('datasets/BERT/train_bert_12000_16000.csv', encoding='utf-8', index=False)

time: 5.6 s (started: 2021-08-09 08:20:02 +00:00)


## Train 5

In [ ]:
# Embeddings trainset
embedding_features=transformer_embedding('bert-base-uncased',train_x_list[16000:20000],TFBertModel)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


time: 4min (started: 2021-08-09 08:20:09 +00:00)


In [ ]:
embedding_features.shape

(4000, 26, 768)

time: 4.03 ms (started: 2021-08-09 08:24:09 +00:00)


In [ ]:
train_bert_x = embedding_features[:,0,:]

time: 1.24 ms (started: 2021-08-09 08:24:09 +00:00)


In [ ]:
train_bert_x.shape

(4000, 768)

time: 5.36 ms (started: 2021-08-09 08:24:09 +00:00)


In [ ]:
train_bert_x_df = pd.DataFrame(train_bert_x)

time: 1.08 ms (started: 2021-08-09 08:24:09 +00:00)


In [ ]:
train_bert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.460084,0.163162,-0.181882,0.060891,-0.157661,0.164490,0.202419,0.205703,-0.351563,-0.175193,0.038144,-0.159480,0.054162,0.318366,-0.088084,-0.020423,-0.095896,0.406262,0.196975,-0.195517,0.091520,-0.189675,-0.207902,-0.297339,0.276799,-0.239818,0.121058,-0.049949,0.171218,0.329823,0.099183,0.158492,0.156369,-0.095277,0.144344,-0.515966,0.128827,0.035684,0.021560,0.083715,...,0.143850,-0.279045,0.115839,0.106726,-0.245902,-0.041231,-0.151606,0.133930,-0.003106,0.067134,0.018099,0.375077,-0.072574,-0.156395,0.050126,0.546555,-0.225066,-0.067128,0.256917,0.085061,-0.031370,-0.290609,0.241711,0.034370,-8.567481,-0.254395,-0.079559,0.310237,0.088282,-0.295305,-0.225312,-0.097510,-0.054378,0.252086,0.365814,-0.064798,-0.089531,-0.103452,0.065068,0.025327
1,0.234874,0.227259,-0.148727,0.015107,-0.345592,-0.516552,0.182159,0.380481,0.066711,-0.226350,0.232032,-0.437657,0.053850,0.451066,0.163041,0.101569,-0.148414,0.138218,0.215933,-0.161585,-0.255358,-0.052269,-0.030805,-0.322970,0.039853,0.081228,0.198353,-0.079688,0.462873,0.143511,0.167082,0.224162,-0.444153,-0.397146,-0.082847,-0.101801,0.596936,-0.147903,0.278455,0.301757,...,-0.007777,-0.248435,0.099168,0.171056,-0.667515,-0.193309,-0.388054,0.004379,-0.090151,0.296276,-0.389323,0.581865,0.146669,0.248513,0.249550,0.095103,-0.161831,0.023668,0.104239,-0.175580,0.129890,-0.106601,-0.373371,0.180828,-6.872911,-0.449474,-0.415629,0.487092,-0.365838,-0.462962,-0.078659,-0.524529,0.170121,-0.034854,-0.170118,0.532658,0.083901,-0.638600,0.205398,0.347954
2,-0.215381,0.512613,-0.382492,0.118548,-0.561764,-0.345460,0.203321,0.498591,0.279430,-0.414807,-0.333991,0.024137,0.014418,0.134057,-0.268807,-0.121083,0.267569,0.687453,0.065546,-0.172730,-0.044849,-0.107708,-0.104959,0.333697,0.028999,-0.210556,-0.169103,-0.196606,0.136719,-0.087021,0.235330,0.010690,0.037711,0.069048,-0.053717,-0.649254,0.181443,0.345869,-0.232860,-0.188829,...,0.360399,0.188148,0.133759,1.215690,-0.713148,-0.320947,0.052735,0.290535,0.084683,-0.311394,-0.767063,0.209594,0.070552,0.164136,0.512188,0.611483,-0.027920,0.022625,0.017589,-0.029562,-0.346611,-0.057186,0.359166,-0.100176,-7.492404,-0.266371,-0.424170,-0.004142,0.100963,0.176953,0.233401,-0.285316,0.075802,-0.072558,0.036461,-0.302069,-0.177869,0.116038,0.305904,0.003967
3,-0.377358,0.247247,-0.130909,-0.441788,-0.690112,0.407986,0.719890,0.835675,-0.708372,0.297186,-0.637668,-0.476258,-0.302642,0.734063,-0.266465,0.145413,-0.435323,0.473247,0.421174,-0.161753,0.418242,-0.509793,0.053004,0.118687,0.037591,-0.309006,0.043366,-0.030843,-0.199676,0.022550,-0.380653,0.404074,0.110212,0.230720,0.399118,-0.209396,0.401330,-0.002851,0.114348,0.372544,...,0.493938,-0.464061,-0.022364,0.395879,-0.373335,-0.448593,-0.182228,0.326618,0.137691,-0.363401,-0.347023,0.509660,0.569095,0.226053,-0.011926,0.384323,0.241316,0.433324,0.202862,-0.074542,0.505007,-0.735708,0.293503,0.362166,-6.346844,-0.515645,-0.186212,0.090298,0.502459,-0.177765,0.070974,-0.042591,0.052665,0.009009,0.298568,-0.435722,0.154697,-0.398110,0.036920,0.349323
4,-0.153325,0.097113,0.106691,-0.011020,0.146519,0.090938,0.429060,0.261258,-0.284939,-0.125829,0.094570,-0.044011,0.052749,0.424623,-0.053167,-0.054327,-0.308282,0.279518,0.151080,-0.241964,-0.145928,-0.303664,0.216031,-0.165961,0.154074,0.000834,-0.086199,-0.045608,-0.192858,0.149824,0.003605,0.074158,0.045604,-0.002968,0.168301,-0.224042,0.096100,-0.173835,0.099312,0.198295,...,-0.093564,0.115113,0.008509,-0.079913,-0.178759,-0.050778,-0.223504,-0.073720,-0.087293,0.056157,-0.249542,0.159009,0.149285,0.105199,0.116171,0.108776,0.125044,-0.133947,0.189629,0.018851,-0.017166,-0.210257,-0.052778,0.058198,-8.564170,-0.233692,-0.197141,0.035850,-0.085249,0.031156,0.1

time: 258 ms (started: 2021-08-09 08:24:09 +00:00)


In [ ]:
train_bert_x_df['is_sarcastic'] = train_y_list[16000:20000]

time: 4.16 ms (started: 2021-08-09 08:24:10 +00:00)


In [ ]:
train_bert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,is_sarcastic
0,-0.460084,0.163162,-0.181882,0.060891,-0.157661,0.164490,0.202419,0.205703,-0.351563,-0.175193,0.038144,-0.159480,0.054162,0.318366,-0.088084,-0.020423,-0.095896,0.406262,0.196975,-0.195517,0.091520,-0.189675,-0.207902,-0.297339,0.276799,-0.239818,0.121058,-0.049949,0.171218,0.329823,0.099183,0.158492,0.156369,-0.095277,0.144344,-0.515966,0.128827,0.035684,0.021560,0.083715,...,-0.279045,0.115839,0.106726,-0.245902,-0.041231,-0.151606,0.133930,-0.003106,0.067134,0.018099,0.375077,-0.072574,-0.156395,0.050126,0.546555,-0.225066,-0.067128,0.256917,0.085061,-0.031370,-0.290609,0.241711,0.034370,-8.567481,-0.254395,-0.079559,0.310237,0.088282,-0.295305,-0.225312,-0.097510,-0.054378,0.252086,0.365814,-0.064798,-0.089531,-0.103452,0.065068,0.025327,1
1,0.234874,0.227259,-0.148727,0.015107,-0.345592,-0.516552,0.182159,0.380481,0.066711,-0.226350,0.232032,-0.437657,0.053850,0.451066,0.163041,0.101569,-0.148414,0.138218,0.215933,-0.161585,-0.255358,-0.052269,-0.030805,-0.322970,0.039853,0.081228,0.198353,-0.079688,0.462873,0.143511,0.167082,0.224162,-0.444153,-0.397146,-0.082847,-0.101801,0.596936,-0.147903,0.278455,0.301757,...,-0.248435,0.099168,0.171056,-0.667515,-0.193309,-0.388054,0.004379,-0.090151,0.296276,-0.389323,0.581865,0.146669,0.248513,0.249550,0.095103,-0.161831,0.023668,0.104239,-0.175580,0.129890,-0.106601,-0.373371,0.180828,-6.872911,-0.449474,-0.415629,0.487092,-0.365838,-0.462962,-0.078659,-0.524529,0.170121,-0.034854,-0.170118,0.532658,0.083901,-0.638600,0.205398,0.347954,0
2,-0.215381,0.512613,-0.382492,0.118548,-0.561764,-0.345460,0.203321,0.498591,0.279430,-0.414807,-0.333991,0.024137,0.014418,0.134057,-0.268807,-0.121083,0.267569,0.687453,0.065546,-0.172730,-0.044849,-0.107708,-0.104959,0.333697,0.028999,-0.210556,-0.169103,-0.196606,0.136719,-0.087021,0.235330,0.010690,0.037711,0.069048,-0.053717,-0.649254,0.181443,0.345869,-0.232860,-0.188829,...,0.188148,0.133759,1.215690,-0.713148,-0.320947,0.052735,0.290535,0.084683,-0.311394,-0.767063,0.209594,0.070552,0.164136,0.512188,0.611483,-0.027920,0.022625,0.017589,-0.029562,-0.346611,-0.057186,0.359166,-0.100176,-7.492404,-0.266371,-0.424170,-0.004142,0.100963,0.176953,0.233401,-0.285316,0.075802,-0.072558,0.036461,-0.302069,-0.177869,0.116038,0.305904,0.003967,0
3,-0.377358,0.247247,-0.130909,-0.441788,-0.690112,0.407986,0.719890,0.835675,-0.708372,0.297186,-0.637668,-0.476258,-0.302642,0.734063,-0.266465,0.145413,-0.435323,0.473247,0.421174,-0.161753,0.418242,-0.509793,0.053004,0.118687,0.037591,-0.309006,0.043366,-0.030843,-0.199676,0.022550,-0.380653,0.404074,0.110212,0.230720,0.399118,-0.209396,0.401330,-0.002851,0.114348,0.372544,...,-0.464061,-0.022364,0.395879,-0.373335,-0.448593,-0.182228,0.326618,0.137691,-0.363401,-0.347023,0.509660,0.569095,0.226053,-0.011926,0.384323,0.241316,0.433324,0.202862,-0.074542,0.505007,-0.735708,0.293503,0.362166,-6.346844,-0.515645,-0.186212,0.090298,0.502459,-0.177765,0.070974,-0.042591,0.052665,0.009009,0.298568,-0.435722,0.154697,-0.398110,0.036920,0.349323,0
4,-0.153325,0.097113,0.106691,-0.011020,0.146519,0.090938,0.429060,0.261258,-0.284939,-0.125829,0.094570,-0.044011,0.052749,0.424623,-0.053167,-0.054327,-0.308282,0.279518,0.151080,-0.241964,-0.145928,-0.303664,0.216031,-0.165961,0.154074,0.000834,-0.086199,-0.045608,-0.192858,0.149824,0.003605,0.074158,0.045604,-0.002968,0.168301,-0.224042,0.096100,-0.173835,0.099312,0.198295,...,0.115113,0.008509,-0.079913,-0.178759,-0.050778,-0.223504,-0.073720,-0.087293,0.056157,-0.249542,0.159009,0.149285,0.105199,0.116171,0.108776,0.125044,-0.133947,0.189629,0.018851,-0.017166,-0.210257,-0.052778,0.058198,-8.564170,-0.233692,-0.197141,0.035850,-0.085249,0.031156,0.144834,-0.124101,-0.038889,0.22

time: 145 ms (started: 2021-08-09 08:24:10 +00:00)


In [ ]:
train_bert_x_df.to_csv('datasets/BERT/train_bert_16000_20000.csv', encoding='utf-8', index=False)

time: 5.57 s (started: 2021-08-09 08:24:10 +00:00)


## Train 6

In [ ]:
# Embeddings trainset
embedding_features=transformer_embedding('bert-base-uncased',train_x_list[20000:],TFBertModel)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


time: 2min 33s (started: 2021-08-09 08:24:17 +00:00)


In [ ]:
embedding_features.shape

(2702, 24, 768)

time: 13.7 ms (started: 2021-08-09 08:26:50 +00:00)


In [ ]:
train_bert_x = embedding_features[:,0,:]

time: 1.14 ms (started: 2021-08-09 08:26:50 +00:00)


In [ ]:
train_bert_x.shape

(2702, 768)

time: 15 ms (started: 2021-08-09 08:26:50 +00:00)


In [ ]:
train_bert_x_df = pd.DataFrame(train_bert_x)

time: 2.2 ms (started: 2021-08-09 08:26:50 +00:00)


In [ ]:
train_bert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.374636,0.189897,0.226349,-0.175439,-0.006375,-0.069437,0.342125,0.354665,-0.287562,-0.151027,0.514292,-0.199649,-0.080802,0.601081,-0.472394,-0.062992,-0.219896,0.217170,0.347797,-0.227397,0.288615,-0.356266,0.271587,-0.437271,0.045098,-0.020070,-0.198037,0.071644,-0.074620,0.178841,-0.000456,0.148499,-0.063305,-0.134740,-0.162042,-0.361099,0.474700,0.423123,0.133044,0.565100,...,0.056655,0.065683,-0.067275,0.084197,-0.345279,-0.139162,-0.151570,0.163379,0.293193,-0.094546,-0.167981,0.018055,0.408070,0.069444,0.469110,0.390232,0.058343,0.055129,-0.039204,0.119952,0.215308,-0.010413,0.077741,0.523541,-7.302530,-0.221654,-0.128267,-0.386612,0.115350,-0.516676,-0.243079,-0.137252,-0.050330,0.099903,-0.014055,0.034623,0.171707,-0.366739,0.412987,0.115466
1,-0.477136,0.287175,-0.361060,0.221085,0.013859,0.051790,0.386204,0.217631,-0.118195,0.070182,-0.272988,-0.330662,0.256327,0.439794,-0.104735,0.125350,-0.264946,0.136278,0.132325,-0.257058,0.246190,-0.449435,-0.204058,-0.158959,-0.103537,0.108911,0.119281,-0.127596,0.153586,0.078325,-0.056814,0.108496,0.069043,-0.311672,0.040420,-0.315078,0.060398,0.042110,0.189714,-0.144185,...,0.053272,0.142179,0.114649,-0.022973,-0.390126,-0.038823,-0.317126,-0.446168,0.074320,0.254256,0.097133,0.213227,0.284486,-0.258732,0.360830,0.452489,-0.043448,0.541599,-0.148496,0.154188,-0.286509,-0.291381,-0.199807,0.363488,-7.813038,0.116303,-0.705744,-0.104424,-0.061625,-0.294135,-0.317811,-0.016843,0.144120,0.232289,-0.095676,-0.032496,-0.116977,-0.279687,0.624516,0.342371
2,-0.464161,-0.272234,0.179644,0.171870,-0.550721,0.162536,0.522825,0.440352,-0.260157,-0.133864,0.379925,-0.032105,0.245603,0.649538,0.167653,-0.060547,-0.349710,0.271557,0.337544,-0.166584,0.131525,-0.813839,-0.236319,0.017598,0.209414,0.089305,-0.179867,-0.158432,0.677215,-0.010605,0.212747,0.469287,-0.357934,-0.403173,0.291970,-0.359523,0.849183,-0.178722,-0.362171,0.117710,...,0.292201,-0.344469,0.130416,0.117914,-0.398580,-0.218944,-0.425618,0.033672,-0.402698,-0.208919,-0.283647,0.646587,-0.016013,-0.292892,-0.080922,0.500197,0.008425,0.277089,0.485952,0.257824,0.097541,-0.573167,0.410088,0.495836,-6.976528,-0.457645,-0.386026,0.115085,-0.012909,-1.015854,-0.319530,-0.227720,-0.119187,-0.134492,0.383846,0.246298,-0.172893,-0.625021,0.078467,0.032596
3,-0.072345,0.034758,0.298465,0.101218,-0.047099,-0.027632,0.594964,-0.201047,-0.310050,-0.393397,-0.130267,-0.537349,-0.075068,0.311770,-0.443460,-0.090509,-0.016956,0.329854,0.170665,0.204137,0.519312,-0.259046,0.311069,-0.079486,-0.271081,-0.391415,0.109258,-0.085458,-0.770901,0.102147,0.079332,0.319838,-0.372978,0.131441,0.410481,-0.046062,-0.174440,-0.098900,0.256868,0.108968,...,-0.108632,-0.413035,0.015906,0.016893,0.005443,-0.135212,-0.184701,-0.072749,0.143323,-0.080082,-0.406723,0.129824,0.526367,0.037421,0.013887,0.385820,0.003058,0.655371,0.100985,0.161565,-0.158176,-0.302322,0.262242,0.204127,-6.051258,-0.067659,-0.105627,0.068908,0.027173,-0.191967,-0.288098,0.109493,0.212927,-0.038023,0.135928,-0.309283,-0.197031,-0.449727,0.146619,0.548404
4,-0.422635,0.269720,-0.039875,-0.069447,-0.245733,-0.119506,0.456885,0.219284,-0.320939,-0.235388,-0.186381,-0.199676,-0.065042,0.325481,0.100667,-0.089147,-0.273246,0.181914,0.246273,-0.136650,0.063330,-0.000701,-0.379463,-0.257089,0.352169,-0.089846,0.105310,0.229455,-0.041861,-0.057001,-0.120556,0.040860,-0.098967,0.132476,-0.000614,-0.189301,0.263677,-0.071936,0.184773,-0.035199,...,0.022909,0.018251,-0.134367,0.217626,-0.214941,0.010755,-0.161858,-0.315471,-0.310620,-0.116734,0.149715,0.168444,0.252148,-0.069823,0.295764,0.389263,0.323053,0.122019,0.013938,0.043682,-0.006352,-0.190255,-0.048727,0.375139,-8.140969,-0.085495,-0.192943,0.140974,0.067

time: 145 ms (started: 2021-08-09 08:26:50 +00:00)


In [ ]:
train_bert_x_df['is_sarcastic'] = train_y_list[20000:]

time: 4.65 ms (started: 2021-08-09 08:26:51 +00:00)


In [ ]:
train_bert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,is_sarcastic
0,-0.374636,0.189897,0.226349,-0.175439,-0.006375,-0.069437,0.342125,0.354665,-0.287562,-0.151027,0.514292,-0.199649,-0.080802,0.601081,-0.472394,-0.062992,-0.219896,0.217170,0.347797,-0.227397,0.288615,-0.356266,0.271587,-0.437271,0.045098,-0.020070,-0.198037,0.071644,-0.074620,0.178841,-0.000456,0.148499,-0.063305,-0.134740,-0.162042,-0.361099,0.474700,0.423123,0.133044,0.565100,...,0.065683,-0.067275,0.084197,-0.345279,-0.139162,-0.151570,0.163379,0.293193,-0.094546,-0.167981,0.018055,0.408070,0.069444,0.469110,0.390232,0.058343,0.055129,-0.039204,0.119952,0.215308,-0.010413,0.077741,0.523541,-7.302530,-0.221654,-0.128267,-0.386612,0.115350,-0.516676,-0.243079,-0.137252,-0.050330,0.099903,-0.014055,0.034623,0.171707,-0.366739,0.412987,0.115466,1
1,-0.477136,0.287175,-0.361060,0.221085,0.013859,0.051790,0.386204,0.217631,-0.118195,0.070182,-0.272988,-0.330662,0.256327,0.439794,-0.104735,0.125350,-0.264946,0.136278,0.132325,-0.257058,0.246190,-0.449435,-0.204058,-0.158959,-0.103537,0.108911,0.119281,-0.127596,0.153586,0.078325,-0.056814,0.108496,0.069043,-0.311672,0.040420,-0.315078,0.060398,0.042110,0.189714,-0.144185,...,0.142179,0.114649,-0.022973,-0.390126,-0.038823,-0.317126,-0.446168,0.074320,0.254256,0.097133,0.213227,0.284486,-0.258732,0.360830,0.452489,-0.043448,0.541599,-0.148496,0.154188,-0.286509,-0.291381,-0.199807,0.363488,-7.813038,0.116303,-0.705744,-0.104424,-0.061625,-0.294135,-0.317811,-0.016843,0.144120,0.232289,-0.095676,-0.032496,-0.116977,-0.279687,0.624516,0.342371,1
2,-0.464161,-0.272234,0.179644,0.171870,-0.550721,0.162536,0.522825,0.440352,-0.260157,-0.133864,0.379925,-0.032105,0.245603,0.649538,0.167653,-0.060547,-0.349710,0.271557,0.337544,-0.166584,0.131525,-0.813839,-0.236319,0.017598,0.209414,0.089305,-0.179867,-0.158432,0.677215,-0.010605,0.212747,0.469287,-0.357934,-0.403173,0.291970,-0.359523,0.849183,-0.178722,-0.362171,0.117710,...,-0.344469,0.130416,0.117914,-0.398580,-0.218944,-0.425618,0.033672,-0.402698,-0.208919,-0.283647,0.646587,-0.016013,-0.292892,-0.080922,0.500197,0.008425,0.277089,0.485952,0.257824,0.097541,-0.573167,0.410088,0.495836,-6.976528,-0.457645,-0.386026,0.115085,-0.012909,-1.015854,-0.319530,-0.227720,-0.119187,-0.134492,0.383846,0.246298,-0.172893,-0.625021,0.078467,0.032596,0
3,-0.072345,0.034758,0.298465,0.101218,-0.047099,-0.027632,0.594964,-0.201047,-0.310050,-0.393397,-0.130267,-0.537349,-0.075068,0.311770,-0.443460,-0.090509,-0.016956,0.329854,0.170665,0.204137,0.519312,-0.259046,0.311069,-0.079486,-0.271081,-0.391415,0.109258,-0.085458,-0.770901,0.102147,0.079332,0.319838,-0.372978,0.131441,0.410481,-0.046062,-0.174440,-0.098900,0.256868,0.108968,...,-0.413035,0.015906,0.016893,0.005443,-0.135212,-0.184701,-0.072749,0.143323,-0.080082,-0.406723,0.129824,0.526367,0.037421,0.013887,0.385820,0.003058,0.655371,0.100985,0.161565,-0.158176,-0.302322,0.262242,0.204127,-6.051258,-0.067659,-0.105627,0.068908,0.027173,-0.191967,-0.288098,0.109493,0.212927,-0.038023,0.135928,-0.309283,-0.197031,-0.449727,0.146619,0.548404,1
4,-0.422635,0.269720,-0.039875,-0.069447,-0.245733,-0.119506,0.456885,0.219284,-0.320939,-0.235388,-0.186381,-0.199676,-0.065042,0.325481,0.100667,-0.089147,-0.273246,0.181914,0.246273,-0.136650,0.063330,-0.000701,-0.379463,-0.257089,0.352169,-0.089846,0.105310,0.229455,-0.041861,-0.057001,-0.120556,0.040860,-0.098967,0.132476,-0.000614,-0.189301,0.263677,-0.071936,0.184773,-0.035199,...,0.018251,-0.134367,0.217626,-0.214941,0.010755,-0.161858,-0.315471,-0.310620,-0.116734,0.149715,0.168444,0.252148,-0.069823,0.295764,0.389263,0.323053,0.122019,0.013938,0.043682,-0.006352,-0.190255,-0.048727,0.375139,-8.140969,-0.085495,-0.192943,0.140974,0.067898,0.069665,-0.368882,0.1650

time: 140 ms (started: 2021-08-09 08:26:51 +00:00)


In [ ]:
train_bert_x_df.to_csv('datasets/BERT/train_bert_20000_end.csv', encoding='utf-8', index=False)

time: 3.83 s (started: 2021-08-09 08:26:51 +00:00)


## Test 1

In [ ]:
'''
Embedding testset

'''
test_x_list = test_x.tolist()
test_y_list = test_y.to_list()

time: 3.16 ms (started: 2021-08-09 08:26:55 +00:00)


In [ ]:
len(test_x_list)

4007

time: 4.95 ms (started: 2021-08-09 08:26:55 +00:00)


In [ ]:
len(test_y_list)

4007

time: 5.81 ms (started: 2021-08-09 08:26:55 +00:00)


In [ ]:
# Embeddings trainset
embedding_features=transformer_embedding('bert-base-uncased',test_x_list,TFBertModel)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


time: 3min 34s (started: 2021-08-09 08:26:55 +00:00)


In [ ]:
embedding_features.shape

(4007, 23, 768)

time: 13.3 ms (started: 2021-08-09 08:30:29 +00:00)


In [ ]:
test_bert_x = embedding_features[:,0,:]

time: 1.21 ms (started: 2021-08-09 08:30:29 +00:00)


In [ ]:
test_bert_x.shape

(4007, 768)

time: 11.3 ms (started: 2021-08-09 08:30:29 +00:00)


In [ ]:
test_bert_x_df = pd.DataFrame(test_bert_x)

time: 4.02 ms (started: 2021-08-09 08:30:29 +00:00)


In [ ]:
test_bert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,0.107940,0.127326,0.268592,0.041054,-0.188099,0.011479,0.601802,0.163164,-0.185784,-0.178802,-0.179043,-0.628296,0.183634,0.627586,-0.139846,-0.107423,-0.295523,0.554930,-0.086268,-0.104365,-0.141416,-0.595281,0.360557,-0.152811,-0.034395,0.030244,0.140431,-0.333595,0.106984,-0.030860,0.136054,0.229988,-0.199928,-0.014378,0.298612,-0.272335,0.115662,0.056998,0.039124,0.344005,...,-0.027178,0.229037,-0.133021,0.431879,-0.234486,0.098132,-0.639773,-0.331548,0.058249,0.004606,-0.209249,-0.048175,0.244342,0.051896,0.108937,0.172387,0.067610,0.252923,0.216486,-0.108399,0.108697,-0.361230,-0.088254,0.054046,-7.835444,0.175229,-0.254698,-0.019120,-0.097172,-0.095873,0.162841,0.028995,-0.081222,-0.020356,0.181344,-0.137660,-0.020104,-0.336589,0.180523,0.076551
1,-0.282037,-0.019341,-0.257521,0.086824,-0.724590,0.069731,0.428775,0.599579,-0.167166,-0.349165,-0.165683,-0.584014,0.030715,0.826014,0.160946,-0.024879,-0.276366,0.542852,0.073964,0.078375,0.274915,-0.383238,0.075774,-0.025499,0.125610,-0.485308,-0.453589,0.074377,-0.064444,0.134514,-0.125840,-0.141922,-0.226371,0.268089,0.437766,-0.623206,0.306493,0.045834,-0.088121,0.462536,...,0.308625,-0.387251,0.279744,0.086036,-0.328647,-0.298855,-0.469013,0.335345,0.151651,0.016252,-0.506489,0.558017,0.306695,0.297551,-0.056090,-0.147299,0.720226,0.511273,-0.318134,-0.132885,0.207322,-0.479583,0.230991,0.229991,-6.006855,-0.288711,-0.380193,0.193862,0.323198,-0.559836,-0.193816,-0.157714,-0.032270,0.055445,0.245763,-0.157119,0.206441,-0.503066,0.430263,0.223601
2,-0.735360,-0.271005,-0.020413,-0.025452,0.372772,-0.330896,0.406013,0.847680,-0.701307,0.035238,-0.074953,-0.335160,0.590062,0.567557,-0.235761,-0.042845,-0.360370,0.120295,0.286335,-0.145904,0.221086,-0.836688,0.196561,0.040481,-0.037968,-0.258488,0.106272,0.207153,-0.209303,-0.074024,-0.081275,0.355968,-0.216797,-0.515447,0.800320,-0.386516,0.630443,-0.751425,-0.310702,0.459443,...,0.020118,-0.168008,0.481048,0.156812,-0.339620,-0.168659,-0.518602,0.137548,0.440760,-0.210898,-0.216903,0.051672,0.131403,-0.585913,0.502704,0.235534,-0.188970,0.542937,0.529399,0.025258,-0.101102,-0.240125,0.323952,0.007018,-6.855280,-0.569735,-0.493719,0.174387,0.095258,-0.659340,0.064014,-0.232742,0.134814,0.220765,0.339001,-0.179042,0.235523,0.017858,0.215072,0.349393
3,-0.445481,0.160300,0.338873,-0.192832,-0.257279,-0.054234,0.713991,0.376818,-0.325833,-0.009373,0.127372,-0.125197,0.023672,0.794231,-0.249243,0.378740,-0.475372,0.318159,0.258193,-0.135755,0.277769,-0.300083,0.197905,-0.058561,0.241780,-0.320792,0.142185,0.106761,0.044768,-0.085482,-0.057003,0.430983,-0.097892,-0.196637,0.383034,-0.417053,0.373758,0.131417,0.120829,0.642791,...,0.214626,-0.204463,0.187959,-0.118497,-0.460877,-0.042954,-0.042918,-0.060312,0.402196,0.165431,0.121028,0.458069,0.297375,-0.277498,0.166741,0.556547,-0.020759,0.394014,0.160022,0.010174,0.374184,-0.061713,0.014172,0.612074,-6.524722,-0.536995,-0.489216,0.228710,-0.099556,-0.618218,-0.005143,-0.284952,0.444610,0.205630,0.398637,0.100465,0.108036,-0.477929,0.465422,0.307502
4,-0.411752,0.104325,-0.011243,0.219443,-0.181268,-0.116425,0.372141,0.003098,-0.629635,-0.470312,0.005463,-0.216234,-0.273368,0.108754,0.002353,-0.161274,0.215756,0.588795,0.224589,0.194519,-0.030456,-1.472353,0.192360,-0.020192,-0.089309,-0.335584,0.037970,0.572849,-0.393251,-0.158871,0.080625,0.715341,-0.535996,-0.700011,0.280994,-0.440321,0.001326,-0.204009,0.103928,0.417648,...,-0.008661,-0.673657,0.224581,0.192383,-0.611154,-0.559788,-0.138280,0.087213,0.573732,0.012575,-0.791673,0.169521,0.274002,-0.300885,0.062256,-0.239751,0.224221,0.936778,-0.073994,0.109853,-0.114307,-0.050425,-0.131086,0.294429,-5.483493,-0.080279,-0.225066,-0.394661,0.044450,-0.791

time: 144 ms (started: 2021-08-09 08:30:29 +00:00)


In [ ]:
test_bert_x_df['is_sarcastic'] = test_y_list

time: 9.01 ms (started: 2021-08-09 08:30:29 +00:00)


In [ ]:
test_bert_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,is_sarcastic
0,0.107940,0.127326,0.268592,0.041054,-0.188099,0.011479,0.601802,0.163164,-0.185784,-0.178802,-0.179043,-0.628296,0.183634,0.627586,-0.139846,-0.107423,-0.295523,0.554930,-0.086268,-0.104365,-0.141416,-0.595281,0.360557,-0.152811,-0.034395,0.030244,0.140431,-0.333595,0.106984,-0.030860,0.136054,0.229988,-0.199928,-0.014378,0.298612,-0.272335,0.115662,0.056998,0.039124,0.344005,...,0.229037,-0.133021,0.431879,-0.234486,0.098132,-0.639773,-0.331548,0.058249,0.004606,-0.209249,-0.048175,0.244342,0.051896,0.108937,0.172387,0.067610,0.252923,0.216486,-0.108399,0.108697,-0.361230,-0.088254,0.054046,-7.835444,0.175229,-0.254698,-0.019120,-0.097172,-0.095873,0.162841,0.028995,-0.081222,-0.020356,0.181344,-0.137660,-0.020104,-0.336589,0.180523,0.076551,0
1,-0.282037,-0.019341,-0.257521,0.086824,-0.724590,0.069731,0.428775,0.599579,-0.167166,-0.349165,-0.165683,-0.584014,0.030715,0.826014,0.160946,-0.024879,-0.276366,0.542852,0.073964,0.078375,0.274915,-0.383238,0.075774,-0.025499,0.125610,-0.485308,-0.453589,0.074377,-0.064444,0.134514,-0.125840,-0.141922,-0.226371,0.268089,0.437766,-0.623206,0.306493,0.045834,-0.088121,0.462536,...,-0.387251,0.279744,0.086036,-0.328647,-0.298855,-0.469013,0.335345,0.151651,0.016252,-0.506489,0.558017,0.306695,0.297551,-0.056090,-0.147299,0.720226,0.511273,-0.318134,-0.132885,0.207322,-0.479583,0.230991,0.229991,-6.006855,-0.288711,-0.380193,0.193862,0.323198,-0.559836,-0.193816,-0.157714,-0.032270,0.055445,0.245763,-0.157119,0.206441,-0.503066,0.430263,0.223601,1
2,-0.735360,-0.271005,-0.020413,-0.025452,0.372772,-0.330896,0.406013,0.847680,-0.701307,0.035238,-0.074953,-0.335160,0.590062,0.567557,-0.235761,-0.042845,-0.360370,0.120295,0.286335,-0.145904,0.221086,-0.836688,0.196561,0.040481,-0.037968,-0.258488,0.106272,0.207153,-0.209303,-0.074024,-0.081275,0.355968,-0.216797,-0.515447,0.800320,-0.386516,0.630443,-0.751425,-0.310702,0.459443,...,-0.168008,0.481048,0.156812,-0.339620,-0.168659,-0.518602,0.137548,0.440760,-0.210898,-0.216903,0.051672,0.131403,-0.585913,0.502704,0.235534,-0.188970,0.542937,0.529399,0.025258,-0.101102,-0.240125,0.323952,0.007018,-6.855280,-0.569735,-0.493719,0.174387,0.095258,-0.659340,0.064014,-0.232742,0.134814,0.220765,0.339001,-0.179042,0.235523,0.017858,0.215072,0.349393,0
3,-0.445481,0.160300,0.338873,-0.192832,-0.257279,-0.054234,0.713991,0.376818,-0.325833,-0.009373,0.127372,-0.125197,0.023672,0.794231,-0.249243,0.378740,-0.475372,0.318159,0.258193,-0.135755,0.277769,-0.300083,0.197905,-0.058561,0.241780,-0.320792,0.142185,0.106761,0.044768,-0.085482,-0.057003,0.430983,-0.097892,-0.196637,0.383034,-0.417053,0.373758,0.131417,0.120829,0.642791,...,-0.204463,0.187959,-0.118497,-0.460877,-0.042954,-0.042918,-0.060312,0.402196,0.165431,0.121028,0.458069,0.297375,-0.277498,0.166741,0.556547,-0.020759,0.394014,0.160022,0.010174,0.374184,-0.061713,0.014172,0.612074,-6.524722,-0.536995,-0.489216,0.228710,-0.099556,-0.618218,-0.005143,-0.284952,0.444610,0.205630,0.398637,0.100465,0.108036,-0.477929,0.465422,0.307502,1
4,-0.411752,0.104325,-0.011243,0.219443,-0.181268,-0.116425,0.372141,0.003098,-0.629635,-0.470312,0.005463,-0.216234,-0.273368,0.108754,0.002353,-0.161274,0.215756,0.588795,0.224589,0.194519,-0.030456,-1.472353,0.192360,-0.020192,-0.089309,-0.335584,0.037970,0.572849,-0.393251,-0.158871,0.080625,0.715341,-0.535996,-0.700011,0.280994,-0.440321,0.001326,-0.204009,0.103928,0.417648,...,-0.673657,0.224581,0.192383,-0.611154,-0.559788,-0.138280,0.087213,0.573732,0.012575,-0.791673,0.169521,0.274002,-0.300885,0.062256,-0.239751,0.224221,0.936778,-0.073994,0.109853,-0.114307,-0.050425,-0.131086,0.294429,-5.483493,-0.080279,-0.225066,-0.394661,0.044450,-0.791892,0.197309,-0.415952,0.03993

time: 142 ms (started: 2021-08-09 08:30:29 +00:00)


In [ ]:
test_bert_x_df.to_csv('datasets/BERT/test_bert.csv', encoding='utf-8', index=False)

time: 5.64 s (started: 2021-08-09 08:30:29 +00:00)


# Merge Trainset


In [ ]:
train_bert_1 = pd.read_csv("datasets/BERT/train_bert_0_4000.csv") 
train_bert_2 = pd.read_csv("datasets/BERT/train_bert_4000_8000.csv")
train_bert_3 = pd.read_csv("datasets/BERT/train_bert_8000_12000.csv") 
train_bert_4 = pd.read_csv("datasets/BERT/train_bert_12000_16000.csv")
train_bert_5 = pd.read_csv("datasets/BERT/train_bert_16000_20000.csv")
train_bert_6 = pd.read_csv("datasets/BERT/train_bert_20000_end.csv")    

time: 11.4 s (started: 2021-08-09 08:30:36 +00:00)


In [ ]:
train_bert_df = train_bert_1.append([train_bert_2, train_bert_3, train_bert_4, train_bert_5, train_bert_6])

time: 56.5 ms (started: 2021-08-09 08:30:48 +00:00)


In [ ]:
train_bert_df.shape

(22702, 769)

time: 6.51 ms (started: 2021-08-09 08:30:48 +00:00)


In [ ]:
train_bert_df.to_csv('datasets/BERT/train_bert.csv', encoding='utf-8', index=False)

time: 45.2 s (started: 2021-08-09 08:30:48 +00:00)


# Statistical Models

In [ ]:
train_bert_df = pd.read_csv("datasets/BERT/train_bert.csv") 
train_bert_df.shape 

(22702, 769)

time: 5.58 s (started: 2021-08-09 08:51:12 +00:00)


In [ ]:
train_bert_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,is_sarcastic
0,-0.190443,0.249115,0.230637,0.046937,-0.425034,-0.272725,0.469147,0.277338,0.008943,-0.050652,-0.017815,-0.104479,0.073786,-0.012094,-0.111905,0.241476,-0.263029,0.073332,0.344830,-0.280562,0.107353,0.028069,-0.080546,-0.326237,0.038039,-0.017133,-0.000988,-0.234635,0.077146,0.459631,0.171129,0.152891,-0.028653,-0.207002,-0.061835,-0.133595,0.480229,0.254105,0.224409,0.312456,...,-0.202688,-0.070961,0.208641,-0.240333,-0.106162,-0.066628,0.101141,0.057052,0.007349,0.063804,0.365955,0.294087,-0.070599,0.379931,0.069376,0.336008,-0.118725,-0.082539,-0.081337,-0.055999,-0.161743,-0.017171,0.261253,-7.710991,-0.120480,-0.371195,-0.246646,0.074852,-0.224048,-0.314523,-0.169071,-0.029449,0.115963,-0.030112,0.272454,0.080770,-0.199147,0.484466,0.361765,0
1,-0.492397,-0.027411,-0.332466,0.020443,-0.610060,-0.148527,0.654926,0.308514,-0.231310,-0.069617,-0.002004,-0.280929,0.227047,0.351792,-0.081066,-0.243597,-0.295800,0.535180,0.217020,-0.006277,-0.016861,-0.292270,0.045566,-0.195326,0.149736,0.058718,0.298146,-0.352757,-0.024628,0.090480,0.199756,0.139113,-0.203298,-0.230459,0.253481,-0.294298,0.042615,0.352720,0.151282,0.150157,...,-0.153747,-0.003750,0.462145,-0.426525,-0.141466,-0.360657,0.167633,0.092457,-0.143379,-0.327042,0.332397,0.360107,0.058052,0.266458,0.623569,0.143433,0.375017,0.051152,0.087787,-0.061374,-0.373077,0.289576,0.027173,-6.620026,-0.146909,-0.239583,0.060715,0.025357,-0.414859,-0.141133,0.228207,-0.018855,0.251574,0.218195,0.030464,-0.055820,-0.330648,0.540078,0.175039,1
2,-0.142799,-0.049289,0.228374,0.020959,-0.200608,-0.045387,0.334416,0.423211,-0.062562,-0.085432,-0.052321,-0.324418,0.136056,0.254049,-0.144795,-0.068938,-0.091523,0.078198,0.236178,-0.176171,-0.107625,-0.274301,-0.115203,-0.137790,0.255695,-0.239128,-0.193295,0.059538,0.189012,0.060158,-0.151148,0.134901,0.110153,0.030933,0.022396,0.065900,-0.086931,-0.064722,-0.112232,-0.054907,...,0.098249,-0.072251,0.135678,-0.098235,-0.133579,-0.321829,-0.142423,0.163555,-0.035497,0.048289,0.128056,0.057226,0.095806,0.247166,0.156255,0.068330,0.002394,0.214267,0.035531,-0.193269,-0.386303,0.063641,-0.027111,-8.786839,-0.177657,-0.019461,-0.183167,-0.132528,-0.192282,0.074134,-0.213986,-0.067644,0.216824,0.095072,-0.164247,-0.103283,-0.315215,0.064099,0.204661,0
3,-0.868000,-0.101078,-0.575666,0.459852,-0.419928,0.016022,0.334006,0.135863,-0.458442,-0.373681,0.098813,-0.304566,-0.724307,0.561535,-0.098482,0.016669,-0.008255,0.168122,-0.026358,-0.096314,0.316348,-0.272942,0.199998,-0.181488,-0.334703,-0.286956,-0.097895,-0.104310,-0.358083,0.068524,-0.019952,-0.093822,-0.243518,-0.114497,0.153138,-0.228161,0.460579,-0.323959,0.125136,0.356893,...,0.274705,-0.066187,0.446192,0.004475,-0.288398,-0.041143,0.137002,0.599168,0.097013,-0.010967,-0.321441,0.173009,-0.415096,0.366950,0.486005,-0.390781,0.732672,0.274839,0.110842,-0.084790,-0.715592,0.140092,-0.168720,-5.533344,-0.255496,-0.490340,-0.269528,0.256371,-0.403477,-0.155346,-0.029823,0.271217,0.259728,0.422689,-0.150633,-0.098891,-0.350663,0.440300,0.144549,1
4,-0.766649,-0.008375,-0.152243,-0.141487,-0.433392,0.113954,0.108358,0.462332,-0.292498,-0.144439,-0.006162,0.005111,-0.093738,0.347403,-0.038388,-0.089119,-0.247187,0.433039,0.025288,-0.218991,0.147774,-0.068135,0.019007,-0.186180,0.399780,0.054598,-0.008111,-0.578480,0.104578,0.158246,0.070361,0.438630,-0.099483,0.172906,-0.088207,-0.365570,0.203475,0.052855,0.113435,0.233232,...,-0.399734,0.280004,0.302165,-0.213661,-0.130853,-0.680508,0.219384,-0.330596,0.184579,-0.553960,0.838544,0.014515,-0.066736,0.215119,0.592264,0.087756,-0.129169,0.353065,0.063154,-0.208683,-0.264319,0.578415,0.125885,-6.732278,-0.223226,-0.731024,-0.099794,0.313067,-0.231150,0.110340,0.

time: 141 ms (started: 2021-08-09 08:51:18 +00:00)


In [ ]:
test_bert_df = pd.read_csv("datasets/BERT/test_bert.csv") 
test_bert_df.shape 

(4007, 769)

time: 1.04 s (started: 2021-08-09 08:51:18 +00:00)


In [ ]:
train_y = train_bert_df.pop('is_sarcastic')
test_y = test_bert_df.pop('is_sarcastic')

train_x = train_bert_df
test_x = test_bert_df

train_x.shape,train_y.shape,test_x.shape,test_y.shape

((22702, 768), (22702,), (4007, 768), (4007,))

time: 14.3 ms (started: 2021-08-09 08:51:19 +00:00)


## SVC

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel ='rbf', gamma = 0.1, C = 10, probability=True) #2h 56min
svc.fit(train_x,train_y)
pred=svc.predict(test_x)

time: 2h 58min 5s (started: 2021-08-09 08:51:19 +00:00)


In [ ]:
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[1990  258]
 [ 664 1095]]
Accuracy Score for LR with C=1.0  =0.7699026703269278
time: 15.6 ms (started: 2021-08-09 11:49:25 +00:00)


## Logistic Regression

In [ ]:
# Applying Logistic Regression on split tfidf baseline
model=LogisticRegression(max_iter=10000)
model.fit(train_x,train_y)
pred=model.predict(test_x)
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[1860  388]
 [ 461 1298]]
Accuracy Score for LR with C=1.0  =0.7881207886199152
time: 25.4 s (started: 2021-08-09 11:49:25 +00:00)


## KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 25, weights = 'distance')
knn.fit(train_x,train_y)
pred=knn.predict(test_x)

time: 2min 48s (started: 2021-08-09 11:49:50 +00:00)


In [ ]:
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[1805  443]
 [ 541 1218]]
Accuracy Score for LR with C=1.0  =0.7544297479411031
time: 10.5 ms (started: 2021-08-09 11:52:39 +00:00)


## Decision Tree

In [ ]:
dt = DecisionTreeClassifier(criterion ='entropy', max_depth= None, min_samples_leaf=1, min_samples_split=20,splitter="best")
dt.fit(train_x,train_y)
pred=dt.predict(test_x)

time: 48.9 s (started: 2021-08-09 11:52:39 +00:00)


In [ ]:
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[1534  714]
 [ 771  988]]
Accuracy Score for LR with C=1.0  =0.6293985525330671
time: 9.04 ms (started: 2021-08-09 11:53:28 +00:00)


## Random Forest

In [ ]:
rf = RandomForestClassifier(criterion='entropy',n_estimators = 100, max_depth = None, bootstrap = False)
rf.fit(train_x,train_y)
pred=rf.predict(test_x)

time: 3min 8s (started: 2021-08-09 11:53:28 +00:00)


In [ ]:
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[1925  323]
 [ 720 1039]]
Accuracy Score for LR with C=1.0  =0.7397055153481408
time: 10.5 ms (started: 2021-08-09 11:56:37 +00:00)


## Naive Bayes

In [ ]:
gnb = GaussianNB()
gnb.fit(train_x,train_y)
pred=gnb.predict(test_x)

time: 537 ms (started: 2021-08-09 11:56:37 +00:00)


In [ ]:
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[1581  667]
 [ 585 1174]]
Accuracy Score for LR with C=1.0  =0.6875467931120539
time: 15.4 ms (started: 2021-08-09 11:56:37 +00:00)
